In [1]:
!pip install requests
!pip install lxml
!pip install selenium
!pip install unidecode
!pip install beautifulsoup4

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 25.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------------------------ --------------- 6.0/9.7 MB 31.0 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.7 MB 20.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 18.8 MB/s eta 0:00:00


In [32]:
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import requests
import lxml as lxml
import html as html
import time
import pandas as pd
import numpy as np 
import re

#Link a la pagina web
link = 'https://diaonline.supermercadosdia.com.ar/'
link_sin_barra = 'https://diaonline.supermercadosdia.com.ar'

In [33]:
# Inicializa el driver de Selenium (asegúrate de tener el driver correspondiente instalado)
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()

#Abre la página web en el navegador controlado por Selenium
driver.get(link)

#Hacemos click en el boton de categorias
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'diaio-custom-mega-menu-0-x-custom-mega-menu-trigger__button')))
button.click()

# Obtiene el HTML después de cargar todos los elementos
html_home = driver.page_source

# Cierra el navegador controlado por Selenium
driver.quit()

#Se busca dentro del HTML las categorias donde hacer hover
sp_home = BeautifulSoup(html_home, 'lxml')
sp_categorias = sp_home.find("div",{"class":"diaio-custom-mega-menu-0-x-category-list__container"}).find_all("a")
categorias = []
for cat in sp_categorias:
    # href = cat.get("href")
    categorias.append(cat.get("href"))
categorias 



['https://diaonline.supermercadosdia.com.ar/almacen',
 'https://diaonline.supermercadosdia.com.ar/bebidas',
 'https://diaonline.supermercadosdia.com.ar/frescos',
 'https://diaonline.supermercadosdia.com.ar/desayuno',
 'https://diaonline.supermercadosdia.com.ar/limpieza',
 'https://diaonline.supermercadosdia.com.ar/perfumeria',
 'https://diaonline.supermercadosdia.com.ar/congelados',
 'https://diaonline.supermercadosdia.com.ar/bebes-y-ninos',
 'https://diaonline.supermercadosdia.com.ar/hogar-y-deco',
 'https://diaonline.supermercadosdia.com.ar/mascotas',
 'https://diaonline.supermercadosdia.com.ar/almacen/golosinas-y-alfajores?PS=15&O=',
 'https://diaonline.supermercadosdia.com.ar/frescos/frutas-y-verduras',
 'https://diaonline.supermercadosdia.com.ar/electro-hogar']

In [ ]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
# Abre la página web en el navegador controlado por Selenium
driver.get(categorias[0])
html_categoria = driver.page_source

#Se obtiene la lista de categorias internas 
sp_categoria_interna = BeautifulSoup(html_categoria, 'lxml')
categorias_internas = sp_categoria_interna.find("div", {"class":"diaio-search-result-0-x-filter__container diaio-search-result-0-x-filter__container--plp bb b--muted-4 diaio-search-result-0-x-filter__container--category-2"})
lista_checkboxes = categorias_internas.find("div", {"class":"diaio-search-result-0-x-filterTemplateOverflow diaio-search-result-0-x-filterTemplateOverflow--plp pb5 overflow-y-auto"}).find_all("input", {"type":"checkbox"})

#Apretar boton pop-up
button_popup = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "vtex-modal__close-icon")))
button_popup.click()

#Lista de categorias dentro de la pagina
id_chechbox = []
for l in lista_checkboxes:
    id_chechbox.append(l.get("id"))
id_chechbox

# Marcamos cada categoría
for i in id_chechbox:
    checkbox = driver.find_element(By.ID, i)
    checkbox.click()
    time.sleep(2)

    elements_count = len(driver.find_elements(By.CLASS_NAME, 'diaio-search-result-0-x-galleryItem'))


Elementos anteriores: 16, Nuevos elementos: 36
No hay nuevos elementos: 36 encontrados.
Elementos anteriores: 16, Nuevos elementos: 36
Elementos anteriores: 36, Nuevos elementos: 38
No hay nuevos elementos: 38 encontrados.


ElementClickInterceptedException: Message: element click intercepted: Element <input type="checkbox" class="vtex-checkbox__input h1 w1 absolute o-0 pointer" id="category-2-arroz-y-legumbres" name="Arroz y Legumbres" value="Arroz y Legumbres" tabindex="0"> is not clickable at point (319, 8). Other element would receive the click: <p class="diaio-custom-top-header-0-x-top_header_text" style="color: rgb(74, 74, 73);">...</p>
  (Session info: chrome=131.0.6778.108)
Stacktrace:
	GetHandleVerifier [0x00007FF7611D6CF5+28821]
	(No symbol) [0x00007FF761143880]
	(No symbol) [0x00007FF760FE578A]
	(No symbol) [0x00007FF761040E8E]
	(No symbol) [0x00007FF76103E92C]
	(No symbol) [0x00007FF76103BAF6]
	(No symbol) [0x00007FF76103AA51]
	(No symbol) [0x00007FF76102CBA0]
	(No symbol) [0x00007FF76105F2FA]
	(No symbol) [0x00007FF76102C3F6]
	(No symbol) [0x00007FF76105F510]
	(No symbol) [0x00007FF76107F412]
	(No symbol) [0x00007FF76105F0A3]
	(No symbol) [0x00007FF76102A778]
	(No symbol) [0x00007FF76102B8E1]


In [4]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
primer_url = link + categorias[0]


# Abre la página web en el navegador controlado por Selenium
driver.get(primer_url)
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))
time.sleep(3)
driver.execute_script("arguments[0].click();", button)
time.sleep(3)

# Obtiene el HTML después de cargar todos los elementos
html_2 = driver.page_source

# Cierra el navegador controlado por Selenium
# Parsea el HTML con BeautifulSoup
soup_2 = BeautifulSoup(html_2, 'lxml')
containers = soup_2.find_all('div', class_='vtex-search-result-3-x-filter__container')
containers


#creamos el filtro que nos devuelve las sub-categorias
filtros = []
sub_categorias_sin_procesar = []
sub_categorias = []

for i in range(len(containers)):
    value = containers[i].get_text()
    filtros.append(value)
    test = re.findall('Sub-Categoría', filtros[i])
    if test:
        texto_lista = containers[i].get_text(separator='\n', strip=True).split('\n')
        sub_categorias_sin_procesar.extend(texto_lista[1:-1]) #Exluimos el ultimo registro

#limpiamos el texto para poder generar las variables que necesitamos
for i in sub_categorias_sin_procesar:
    texto_procesado = unidecode(i).lower()
    texto = texto_procesado.replace(' ','-')
    texto_2 = texto.replace(',', '')
    sub_categorias.append(texto_2)

driver.quit()

for s in range(len(sub_categorias)):
    primer_url = link + categorias[0]
    driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
    driver.maximize_window()
    driver.get(primer_url)
    button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))
    time.sleep(3)
    driver.execute_script("arguments[0].click();", button)
    time.sleep(3)
    selector_sub_categoria = f'label[for="category-3-{sub_categorias[s]}"]' #Creamos la categoria que necesitamos para hacer click
    checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector_sub_categoria)))
    checkbox.click()
    time.sleep(5)
    
    prev_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        curr_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

        if curr_elements_count == prev_elements_count:
            break
        
        # Actualizar el contador de elementos
        prev_elements_count = curr_elements_count

    # Obtiene el HTML después de cargar todos los elementos
    html_3 = driver.page_source
    time.sleep(2)

    # Cierra el navegador controlado por Selenium
    driver.quit()

    soup_3 = BeautifulSoup(html_3, 'lxml')
    productos_categoria = soup_3.find_all('div', class_='vtex-search-result-3-x-galleryItem')
    productos_categoria

    total_items = []

    for litem_listado in range(len(productos_categoria)):
        try:
            # Extraemos link de la imagen del producto
            imagen = productos_categoria[litem_listado].find('img')
            link_imagen = imagen['src'] if imagen else None

            # Extraemos link a la web del producto en el supermercado
            web_producto = productos_categoria[litem_listado].find('a')
            url_producto = link_sin_barra + web_producto['href'] if web_producto else None

            # Datos del producto
            informacion_producto_wrapped = productos_categoria[litem_listado].find('div', 'vtex-flex-layout-0-x-flexCol')
            informacion_producto = informacion_producto_wrapped.find_all('div', 'vtex-flex-layout-0-x-flexColChild') if informacion_producto_wrapped else []

            # Creamos las listas soporte
            datos_productos_sin_procesar = []
            datos_productos_procesados = []
            datos_productos_procesados.append(categorias[0])  # Agregamos la categoria a la que pertenece el producto
            datos_productos_procesados.append(sub_categorias[s])  # Agregamos la categoria a la que pertenece el producto

            # Con este bucle for extraemos la informacion sin procesar
            for info in informacion_producto:
                valor_lista = info.get_text(strip=True).replace('\xa0', ' ')
                datos_productos_sin_procesar.append(valor_lista)

            # Generamos la expresion regular que extrae el precio correcto
            # y conformamos la lista con los datos del producto
            for i in range(0, 4):
                datos_productos_procesados.append(datos_productos_sin_procesar[i] if i < len(datos_productos_sin_procesar) else None)

            for i in range(len(datos_productos_sin_procesar)):
                patron_regex_precios = r"(\$\d{1,3}(?:\.\d{3})*,\d{2})"
                resultado_regex = re.findall(patron_regex_precios, datos_productos_sin_procesar[i])
                if resultado_regex:
                    datos_productos_procesados.extend(resultado_regex)

            if len(datos_productos_procesados) == 7:
                datos_productos_procesados.append(None)
                datos_productos_procesados.append(datos_productos_sin_procesar[-1])
            else:
                datos_productos_procesados.append(datos_productos_sin_procesar[-1])

            datos_productos_procesados.append(url_producto)  # Agregamos el link al producto
            datos_productos_procesados.append(link_imagen)  # Agregamos el link a la imagen

            total_items.append(datos_productos_procesados)
        except Exception as e:
            print(f"Error procesando el producto {litem_listado}: {e}")

    nombre_dataframe = f'data_producto_{categorias[0]}_{sub_categorias[s]}'
    lista_columnas = ('categoria', 'producto', 'banner_producto', 'marca', 'descripcion_producto',
                    'promocion', 'precio', 'precio_sin_descuento', 'precio_por_kilo',
                    'url_producto', 'url_imagen')

    globals()[nombre_dataframe] = pd.DataFrame(total_items, columns=lista_columnas)
    ruta_csv = f'C:/Users/phoen/OneDrive/Escritorio/Lauta/Data Science/Projects/scrapper_dia/Productos obtenidos/{nombre_dataframe}.csv'
    globals()[nombre_dataframe].to_csv(ruta_csv, index=False)

## Next steps

Poder profesionalizar este entorno para que cualquiera pueda acceder
